In [ ]:
import pandas as pd
import numpy as np

import scipy.stats

import ast
import pickle

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns

import matplotlib.lines as mlines
from matplotlib.figure import Figure

import re
import math

In [ ]:
pd.options.display.max_columns = None

# Store workload predictions sem -> cid -> pred

In [ ]:
# Input
fs = [
    '../research-data/processed/course-features-2017 Spring.csv',
    '../research-data/processed/course-features-2017 Fall.csv',
    '../research-data/processed/course-features-2018 Spring.csv',
    '../research-data/processed/course-features-2018 Fall.csv',
    '../research-data/processed/course-features-2019 Spring.csv',
    '../research-data/processed/course-features-2019 Fall.csv',
    '../research-data/processed/course-features-2020 Spring.csv',
    '../research-data/processed/course-features-2020 Fall.csv',
    '../research-data/processed/course-features-2021 Spring.csv'
]

# Sems
refs = [f.split('-')[-1].split('.')[0] for f in fs]

# Output
fs = [f'../research-data/processed/predicted-course-loads-{ref}.csv' for ref in refs]

In [ ]:
dfs = [] 
for f, sem in zip(fs, refs):
    tmp = pd.read_csv(f)
    tmp = tmp[['course_name_number', 'cl_combined', 'secondary_sections']]
    #tmp = tmp.set_index('course_name_number')
    tmp = tmp.drop_duplicates()
    tmp['#SEMESTER_YEAR_NAME_CONCAT'] = sem
    tmp = tmp.rename(columns = {'course_name_number': 'COURSE_SUBJECT_NAME_NUMBER'})
    tmp = tmp[['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER', 'secondary_sections', 'cl_combined']]
    dfs.append(tmp)
df_load = pd.concat(dfs)

In [ ]:
df_load.head()

# Read in grade data

In [ ]:
cols2read = ['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER', 'COURSE_SUBJECT_SHORT_NM', 
             'STUDENT_CREDIT_HRS_NBR', 'GRADE_NM',
             'GRADE_POINTS_NBR', 'GRADE_SORT_NBR', 'GRADE_SUBTYPE_DESC', 'SECTION_NBR',
             'GRADE_TYPE_DESC', 'ANON_ID', 'SNAPSHOT_CODE', 'COURSE_CONTROL_NBR',
             'CRS_ACADEMIC_DEPT_SHORT_NM', 'OFFERING_TYPE_DESC']

In [ ]:
df_grade = pd.read_csv('../edw_askoski_student_grades_hashed.txt', 
                      sep='|', low_memory=False, usecols = cols2read)

# Sample data for cohort that enrolled in fall 2017

In [ ]:
df_cohort = pd.read_csv('../edw_askoski_student_cohorts_hashed.txt', sep='|')

In [ ]:
# Fall 2017 intake
df_cohort_2017 = df_cohort[df_cohort['#SEMESTER_YEAR_NAME_CONCAT'] == '2017 Fall'].copy()

df_cohort_2017 =\
    df_cohort_2017[['ANON_ID', '#SEMESTER_YEAR_NAME_CONCAT', 'YEARS_TO_GRADUATION', 'APPLICATION_ENTRY_TYPE']]

df_cohort_2017.APPLICATION_ENTRY_TYPE.value_counts()

In [ ]:
# Check if all relevant students are in the data set
assert len(set(df_cohort_2017['ANON_ID']) - set(df_grade['ANON_ID'])) == 0, 'MISSING MATCHES'

In [ ]:
# Apply filter
df_grade_orig = df_grade.copy()
df_grade = df_grade[df_grade['ANON_ID'].isin(set(df_cohort_2017['ANON_ID']))].copy() 
df_grade = df_grade[df_grade['#SEMESTER_YEAR_NAME_CONCAT'].isin(set(df_load['#SEMESTER_YEAR_NAME_CONCAT']))].copy()
df_grade = df_grade[df_grade['#SEMESTER_YEAR_NAME_CONCAT'] != '2017 Spring'].copy()
df_grade = df_grade[df_grade['COURSE_SUBJECT_NAME_NUMBER'] != '- -'].copy()

In [ ]:
df_grade.shape

In [ ]:
df_cohort_2017.head()

In [ ]:
df_grade['#SEMESTER_YEAR_NAME_CONCAT'].value_counts()

# Sample only primary sections with secondary sections based on which LMS data is used for workload prediction

In [ ]:
# Step 1: Filter based on course ID
perc_omitted = 1 - sum(df_grade['COURSE_SUBJECT_NAME_NUMBER'].isin(set(df_load['COURSE_SUBJECT_NAME_NUMBER'])))/df_grade.shape[0]
df_grade_analysis = df_grade[df_grade['COURSE_SUBJECT_NAME_NUMBER'].isin(set(df_load['COURSE_SUBJECT_NAME_NUMBER']))].copy()
print(f'Step 1: {round(perc_omitted*100, 2)}% omitted')

In [ ]:
# Step 2: If Semester x Course is available (no mean imputing), filter semester-level secondary sections
#         Else, filter all available LMS secondary sections across semesters

# Dict with course id ->  all secondary sections
def eval_list_with_nan(s):
    s = s.replace('nan, ', '')
    try:
        res = ast.literal_eval(s)
    except:
        res = []
    return res

df_load['secondary_sections_list'] = df_load['secondary_sections'].map(eval_list_with_nan)

cross_semester_secondary_sections = df_load\
    .groupby('COURSE_SUBJECT_NAME_NUMBER')\
    [['secondary_sections_list']]\
    .sum()

def int_list_to_filled_str_list(l, n_left=3):
    return list(set([str(i).zfill(n_left) for i in l]))

cross_semester_secondary_sections['secondary_sections_list'] =\
    cross_semester_secondary_sections['secondary_sections_list'].map(int_list_to_filled_str_list)

# Add available predictions on a semester basis
df_grade_analysis = df_grade_analysis\
    .merge(df_load[['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER', 'secondary_sections']],
           how = 'outer', on = ['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER'])

# Join generic available LMS secondary sections across semester
df_grade_analysis['secondary_sections_cross_semester'] =\
    df_grade_analysis.COURSE_SUBJECT_NAME_NUMBER.map(cross_semester_secondary_sections.to_dict()['secondary_sections_list'])

# Fill NA of reference variable for grade filtering with cross semester list
df_grade_analysis['secondary_sections'] = df_grade_analysis['secondary_sections'].fillna(df_grade_analysis['secondary_sections_cross_semester'])
del df_grade_analysis['secondary_sections_cross_semester']

# Omit empty entries, around 5000 entries in Fall 2017 intake
df_grade_analysis.dropna(subset=['COURSE_CONTROL_NBR'], inplace=True)

# Apply filter of secondary sections directly based on remaining rows after step 1
def check_overlap_secondary_sections(s: str, l: list):
    if len(s) < 3:
        s = s.zfill(3) # There are some section nums which are not padded
    return s in l

df_grade_analysis = df_grade_analysis[
    df_grade_analysis.apply(lambda x: check_overlap_secondary_sections(x.SECTION_NBR, x.secondary_sections), axis=1)
].copy()

# Filter primary section enrollment data (as were courses rated)

In [ ]:
df_grade = df_grade[df_grade['OFFERING_TYPE_DESC'] == 'Primary'].copy()

# Check match statistics

In [ ]:
len(set(df_grade['COURSE_SUBJECT_NAME_NUMBER']))

In [ ]:
# Check if any course ID can not be matched
len(set(df_grade['COURSE_SUBJECT_NAME_NUMBER']) - set(df_load['COURSE_SUBJECT_NAME_NUMBER'])) /\
    len(set(df_grade['COURSE_SUBJECT_NAME_NUMBER']))

In [ ]:
# Export all course, semester pairs based on primary sections
dout = df_grade[['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER', 'SECTION_NBR']].drop_duplicates()

# LMS completeness statistics -> Add to manuscript maybe
for semester in df_load['#SEMESTER_YEAR_NAME_CONCAT'].unique():
    courses_with_lms = set(df_load[df_load['#SEMESTER_YEAR_NAME_CONCAT'] == semester].COURSE_SUBJECT_NAME_NUMBER)
    courses_taken = set(dout[dout['#SEMESTER_YEAR_NAME_CONCAT'] == semester].COURSE_SUBJECT_NAME_NUMBER)
    try:
        res = round(len(courses_taken&courses_with_lms)/len(courses_taken),4)*100
    except:
        res = ''
    print(f'In semester {semester}, {res}%')
    
courses_with_lms = set(df_load.COURSE_SUBJECT_NAME_NUMBER)
courses_taken = set(dout.COURSE_SUBJECT_NAME_NUMBER)
res = round(len(courses_taken&courses_with_lms)/len(courses_taken),4)*100
print(f'In all semesters, {res}%')

# Aggregate student GPA per semester

In [ ]:
skip = False
fpath = '../research-data/processed/student-semester-gpas.p'
if not skip:
    #tmp = df_grade.sample(10000).copy()
    #df_gpa = tmp[tmp['GRADE_TYPE_DESC'] == 'Letter Grade']\
    df_gpa = df_grade[(df_grade['GRADE_TYPE_DESC'] == 'Letter Grade') & (df_grade['SNAPSHOT_CODE'] == 'EOT')]\
                    .groupby(['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])\
                    [['GRADE_POINTS_NBR', 'STUDENT_CREDIT_HRS_NBR']]\
                    .apply(lambda grades: np.sum(grades['GRADE_POINTS_NBR'] * grades['STUDENT_CREDIT_HRS_NBR']) / np.sum(grades['STUDENT_CREDIT_HRS_NBR']))\
                    .reset_index()\
                    .rename(columns={0: '#SEMESTER_GPA'})

    with open(fpath, 'wb') as f:
        pickle.dump(df_gpa, f)
else:
    with open(fpath, 'rb') as f:
        df_gpa = pickle.load(f)

# Aggregate STEM status of student within a semester

In [ ]:
d_dept_stem = {
'-': np.nan,
'African American Studies': False,
'Ag & Env Chem Grad Grp': True,
'Ag & Resource Econ & Pol': True,
'Anc Hist Med Arc Grad Grp': False,
'Ancient Greek & Roman Studies': False,
'Anthropology': False,
'Applied Sci & Tech Grad Grp': True,
'Architecture': True,
'Art Practice': False,
'Asian Studies Grad Grp': False,
'Astronomy': True,
'Bioengineering': True,
'Bioengineering-UCSF Grad Grp': True,
'Biophysics Grad Grp': True,
'Biostatistics Grad Grp': True,
'Buddhist Studies Grad Grp': False,
'Business': False,
'Chem & Biomolecular Eng': True,
'Chemistry': True,
'City & Regional Planning': True,
'Civil & Environmental Eng': True,
'Classics': False,
'College Writing Programs': False,
'Comparative Biochem Grad Grp': True,
'Comparative Literature': False,
'Computational Biology Grad Grp': True,
'Critical Theory Grad Grp': False,
'Data Science': True,
'Demography': False,
'Design Innovation': False,
'Development Eng Grad Grp': True,
'Development Practice Grad Grp': False,
'Earth & Planetary Science': True,
'East Asian Lang & Culture': False,
'Economics': True,
'Education': False,
'Electrical Eng & Computer Sci': True,
'Endocrinology Grad Grp': True,
'Energy & Resources Grad Grp': True,
'Engineering Joint Programs': True,
'Engineering Science': True,
'English': False,
'Env Sci, Policy, & Mgmt': True,
'Environmental Health Sci GG': True,
'Epidemiology Grad Grp': True,
'Ethnic Studies': False,
'European Studies Grad Grp': False,
'FPF-African American Studies': False,
'FPF-Anc Greek & Roman Studies': False,
'FPF-Anthropology': False,
'FPF-Art Practice': False,
'FPF-Astronomy': True,
'FPF-Chemistry': True,
'FPF-Classics': False,
'FPF-College Writing Program': False,
'FPF-Comparative Literature': False,
'FPF-Earth & Planetary Science': True,
'FPF-English': False,
'FPF-Env Sci, Policy, & Mgmt': True,
'FPF-Ethnic Studies': False,
'FPF-Film & Media': False,
'FPF-Gender & Womens Studies': False,
'FPF-Geography': False,
'FPF-History': False,
'FPF-History of Art': False,
'FPF-IAS Teaching Program': False,
'FPF-Integrative Biology': True,
'FPF-Interdisc Social Sci Pgms': False,
'FPF-Legal Studies': False,
'FPF-Letters & Science': np.nan, # comprises of multiple departments
'FPF-Linguistics': False,
'FPF-Mathematics': True,
'FPF-Molecular & Cell Biology': True,
'FPF-Music': False,
'FPF-Philosophy': False,
'FPF-Political Science': False,
'FPF-Psychology': True,
'FPF-Rhetoric': False,
'FPF-Sociology': False,
'FPF-South & SE Asian Studies': False,
'FPF-Statistics': True,
'FPF-UG Interdisciplinary Stds': False,
'Film and Media': False,
'Folklore Grad Grp': False,
'French': False,
'Gender & Womens Studies': False,
'Geography': False,
'German': False,
'Global Metro Std Grad Grp': False,
'Global Studies Grad Grp': False,
'Grad Division Other Programs': np.nan,
'Health & Medical Sci Grad Grp': True,
'Health Policy GG': False,
'History': False,
'History of Art': False,
'Industrial Eng & Ops Research': True,
'Infectious Diseases & Immun GG': True,
'Information': True,
'Integrative Biology': True,
'Interdisc Social Science Pgms': False,
'Interdisciplinary Doctoral Pgm': False,
'Italian Studies': False,
'JSP Graduate Program': False,
'Jewish Studies Program': False,
'Journalism': False,
'L&S Chemistry': True,
'L&S Computer Science': True,
'L&S Data Science': True,
'L&S Envir Econ & Policy': True,
'L&S Legal Studies': False,
'L&S Ops Rsch & Mgmt Sci': True,
'L&S Public Health': True,
'L&S Social Welfare': False,
'L&S Undeclared': np.nan,
'Landscape Arch & Env Plan': True,
'Latin American Studies GG': False,
'Law': False,
'Linguistics': False,
'Logic and Method of Science GG': False,
'Materials Science & Eng': True,
'Mathematics': True,
'Mechanical Engineering': True,
'Medieval Studies Program': False,
'Metabolic Biology Grad Grp': True,
'Microbiology Grad Grp': True,
'Middle Eastern Lang & Cultures': False,
'Military Affairs Program': False,
'Molecular & Cell Biology': True,
'Molecular Toxicology Grad Grp': True,
'Music': False,
'Nano Sci & Eng Grad Grp': True,
'Near Eastern Religions GG': False,
'Near Eastern Studies': False,
'Neuroscience Graduate Program': True,
'New Media Grad Grp': False,
'Nuclear Engineering': True,
'Nutritional Sciences & Tox': True,
'Optometry': True,
'Other Arts & Humanities Pgms': False,
'Other Bio Sciences Pgms': True,
'Other Clg of Natural Res Pgms': True,
'Other EVCP Programs': False,
'Other Env Design Programs': True,
'Other Math & Physical Sci Pgms': True,
'Other Social Sciences Programs': False,
'Performance Studies Grad Grp': False,
'Philosophy': False,
'Physical Education': False,
'Physics': True,
'Plant & Microbial Biology': True,
'Political Science': False,
'Psychology': True,
'Public Health': True,
'Public Policy': False,
'Rangeland & Wildlife Mgmt GG': False,
'Rhetoric': False,
'Romance Lang & Lit Grad Pgm': False,
'Scandinavian': False,
'Sci & Tech Stds Grad Grp': True,
'Science & Math Educ Grad Grp': True,
'Slavic Languages & Literatures': False,
'Social Welfare': False,
'Sociology': False,
'Sociology and Demography GG': False,
'South & SE Asian Studies': False,
'Spanish & Portuguese': False,
'Statistics': True,
'Study of Religion Grad Grp': False,
'Theater Dance & Perf Stds': False,
'UC Education Abroad Program': False,
'UCBX-Concurrent Enrollment Dpt': False,
'UG Interdisciplinary Studies': False,
'Urban Design Grad Grp': False,
'Vision Science Grad Grp': True
}

df_majors = pd.read_csv('../edw_askoski_student_majors_hashed.txt', sep='|')

# Filter undeclared entries
df_majors =\
    df_majors[df_majors.MAJOR_NAME.map(lambda s: 'undeclared' not in s.lower())]

# Get anon IDs that switched major
anon_switched = df_majors.groupby('ANON_ID')['MAJOR_NAME'].unique().reset_index().MAJOR_NAME.map(len) > 1
anon_switched = pd.DataFrame(anon_switched).reset_index().rename(columns={'index': 'ANON_ID', 'MAJOR_NAME': 'switched_major'})
df_majors = df_majors.merge(anon_switched, how='left', on='ANON_ID')

# Get first non-undeclared major
df_majors = df_majors.drop_duplicates(subset="ANON_ID")

# Add STEM status of student
df_majors['is_stem_major_student'] =\
    df_majors[['ANON_ID', 'ACADEMIC_DEPARTMENT_NAME']].ACADEMIC_DEPARTMENT_NAME.map(d_dept_stem)
join_this = df_majors[['ANON_ID', 'is_stem_major_student', 'switched_major', 'ACADEMIC_DEPARTMENT_NAME', 'ACADEMIC_DIVISION_NAME']]

df_grade = df_grade.merge(join_this, how = 'left', on = 'ANON_ID')

df_grade['is_stem_course'] = df_grade.CRS_ACADEMIC_DEPT_SHORT_NM.map(d_dept_stem)

In [ ]:
df_grade['student_course_stem_match'] = ~(df_grade['is_stem_course'] ^ df_grade['is_stem_major_student'])

In [ ]:
df_grade.head()

# Aggregate student dropout per semester

In [ ]:
def set_difference(a, b):
    try:
        a = set(a)
    except:
        a = set() # np.nan
    try:
        b = set(b)
    except:
        b = set() # np.nan
    return a-b

def set_union(a, b):
    try:
        a = set(a)
    except:
        a = set() # np.nan
    try:
        b = set(b)
    except:
        b = set() # np.nan
    return a|b

In [ ]:
# any courses that are in CEN but not in EOT should constitute a dropped course

skip = False
fpath = '../research-data/processed/student-semester-dropoutratios.p'

if not skip:
    # any courses that are in CEN but not in EOT should constitute a dropped course
    courses_cen = df_grade\
                        [df_grade['SNAPSHOT_CODE'] == 'CEN']\
                        .groupby(['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])\
                        ['COURSE_SUBJECT_NAME_NUMBER']\
                        .unique()\
                        .reset_index()\
                        .rename(columns={'COURSE_SUBJECT_NAME_NUMBER': 'COURSES_CEN'})

    courses_eot = df_grade\
                        [df_grade['SNAPSHOT_CODE'] == 'EOT']\
                        .groupby(['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])\
                        ['COURSE_SUBJECT_NAME_NUMBER']\
                        .unique()\
                        .reset_index()\
                        .rename(columns={'COURSE_SUBJECT_NAME_NUMBER': 'COURSES_EOT'})
    
    # 2020 Spring only has Week 17 and not CEN
    courses_week_17 = df_grade\
                        [df_grade['SNAPSHOT_CODE'] == 'Week 17']\
                        .groupby(['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])\
                        ['COURSE_SUBJECT_NAME_NUMBER']\
                        .unique()\
                        .reset_index()\
                        .rename(columns={'COURSE_SUBJECT_NAME_NUMBER': 'COURSES_W17'})

    df_dropout = courses_cen\
        .merge(courses_eot, how='left', on=['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])\
        .merge(courses_week_17, how='left', on=['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])

    df_dropout['COURSES_DROPPED'] =\
        df_dropout.apply(lambda row: set_difference(row['COURSES_CEN'], row['COURSES_EOT']), axis=1)

    df_dropout['COURSES_ALL'] =\
        df_dropout.apply(lambda row: set_union(row['COURSES_CEN'], row['COURSES_EOT']), axis=1)
    
    df_dropout['COURSES_ALL'] =\
        df_dropout.apply(lambda row: set_union(row['COURSES_ALL'], row['COURSES_W17']), axis=1)

    df_dropout['RATIO_COURSES_DROPPED'] = df_dropout['COURSES_DROPPED'].map(len) / df_dropout['COURSES_ALL'].map(len)

    with open(fpath, 'wb') as f:
        pickle.dump(df_dropout, f)
else:
    with open(fpath, 'rb') as f:
        df_dropout = pickle.load(f)

# Combine load data sets and calculate average load per student, semester

In [ ]:
df_student_load = df_grade[['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER', 
                            'ANON_ID', 'STUDENT_CREDIT_HRS_NBR', 'SNAPSHOT_CODE', 
                            'is_stem_major_student', 'is_stem_course', 'student_course_stem_match', 
                            'switched_major', 'OFFERING_TYPE_DESC', 'ACADEMIC_DIVISION_NAME',
                            'ACADEMIC_DEPARTMENT_NAME']]\
    .merge(df_load, how='left', on=['#SEMESTER_YEAR_NAME_CONCAT', 'COURSE_SUBJECT_NAME_NUMBER'])

In [ ]:
# Impute by adding average out of all available semesters
d_load2impute = df_load\
    .groupby('COURSE_SUBJECT_NAME_NUMBER')\
    .mean()\
    .reset_index()\
    .set_index('COURSE_SUBJECT_NAME_NUMBER')\
    .to_dict()\
    ['cl_combined']

In [ ]:
df_student_load['cl_combined_sem_avg'] = \
    df_student_load.COURSE_SUBJECT_NAME_NUMBER.map(d_load2impute)

In [ ]:
df_student_load['cl_combined_imputed'] = \
    [a if not pd.isna(a) else b for a, b in zip(df_student_load['cl_combined'], df_student_load['cl_combined_sem_avg'])]

In [ ]:
# How many courses were imputations based on, on average?
nullcourses = set(df_student_load[df_student_load['cl_combined'].isnull()].COURSE_SUBJECT_NAME_NUMBER)
df_student_load[df_student_load['COURSE_SUBJECT_NAME_NUMBER'].isin(nullcourses)][['COURSE_SUBJECT_NAME_NUMBER', 'cl_combined']].dropna().drop_duplicates().groupby('COURSE_SUBJECT_NAME_NUMBER').size().mean()

In [ ]:
df_student_load['cl_combined'].isnull().sum() / df_student_load.shape[0]

In [ ]:
skip = False
fpath = '../research-data/processed/student-semester-loads.p'
if not skip:
    # 2020 Spring only has Week 17 and not CEN
    df_student_load_amended = df_student_load.copy()
    df_student_load_amended.loc[(df_student_load_amended['#SEMESTER_YEAR_NAME_CONCAT']=='2020 Spring') & 
                    (df_student_load_amended['SNAPSHOT_CODE']=='Week 17'), 'SNAPSHOT_CODE'] = 'CEN'
    df_student_load_semester = df_student_load_amended\
                    [df_student_load_amended['SNAPSHOT_CODE'] == 'CEN']\
                    .groupby(['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])\
                    [['cl_combined', 'cl_combined_imputed', 'STUDENT_CREDIT_HRS_NBR', 
                      'is_stem_major_student', 'is_stem_course', 'student_course_stem_match', 
                      'switched_major', 'ACADEMIC_DIVISION_NAME']]\
                    .apply(lambda loads: {'sem_load_credit_hours': np.sum(loads['STUDENT_CREDIT_HRS_NBR']),
                                          'sem_load_predicted': np.sum(loads['cl_combined_imputed']),
                                          'sem_courses': len(loads['cl_combined_imputed']),
                                          'sem_courses_nan_predicted_load': loads['cl_combined'].isna().sum(),
                                          'is_stem_major_student': loads['is_stem_major_student'].values[0],
                                          'switched_major': loads['switched_major'].values[0],
                                          'n_courses': loads.shape[0],
                                          'n_stem_courses': loads['is_stem_course'].sum(),
                                          'n_courses_stem_match': loads['student_course_stem_match'].sum(),
                                          'ratio_courses_stem_match': np.mean(loads['student_course_stem_match']),
                                          'ACADEMIC_DIVISION_NAME': loads['ACADEMIC_DIVISION_NAME'].values[0]})\
                    .reset_index()\
                    .rename(columns={0: '#SEMESTER_LOAD'})
    
    add_cols = df_student_load_semester['#SEMESTER_LOAD'].apply(pd.Series)
    del df_student_load_semester['#SEMESTER_LOAD']
    df_student_load_semester = pd.concat([df_student_load_semester, add_cols], axis=1)
        
    with open(fpath, 'wb') as f:
        pickle.dump(df_student_load_semester, f)
else:
    with open(fpath, 'rb') as f:
        df_student_load_semester = pickle.load(f)

In [ ]:
# ratio missing predictions without imputation
df_student_load_semester['ratio_missing_predictions'] = \
    df_student_load_semester['sem_courses_nan_predicted_load'] / df_student_load_semester['sem_courses']

In [ ]:
df_student_load_semester['ratio_missing_predictions'].mean()

In [ ]:
len(set(df_student_load_semester.ANON_ID))

# Join semester load to semester GPA and dropout

In [ ]:
pd.unique(df_student_load_semester['#SEMESTER_YEAR_NAME_CONCAT'])

In [ ]:
df = df_student_load_semester\
        .merge(df_gpa, how='outer', on=['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])

In [ ]:
df = df.merge(df_dropout, how='outer', on=['#SEMESTER_YEAR_NAME_CONCAT', 'ANON_ID'])

In [ ]:
df.sample(5)

In [ ]:
# JOIN enrollment status variable (Transfer vs non Transfer)
df = df\
    .merge(df_cohort_2017[['ANON_ID', 'YEARS_TO_GRADUATION', 'APPLICATION_ENTRY_TYPE']], 
           how = 'left', on = 'ANON_ID')

In [ ]:
d_code = {
    'NEW FRESHMEN': 'Non-Transfer',
    'ADVANCED STANDING': 'Transfer',
    'SECOND BACHELOR?S DEGREE': 'Non-Transfer'
}

In [ ]:
df['is_transfer'] = df.APPLICATION_ENTRY_TYPE.map(d_code)

# Add semester count and program retention variables

In [ ]:
def get_semester_count(s):
    if s=='2017 Fall': return 1
    elif s=='2018 Spring': return 2
    elif s=='2018 Fall': return 3
    elif s=='2019 Spring': return 4
    elif s=='2019 Fall': return 5
    elif s=='2020 Spring': return 6
    elif s=='2020 Fall': return 7
    elif s=='2021 Spring': return 8
    else: return ''

df['semester_count'] = df['#SEMESTER_YEAR_NAME_CONCAT'].map(get_semester_count)

df['semester_count'].value_counts()

In [ ]:
def get_true_sem_count(row):
    if row['is_transfer'] == 'Transfer':
        return row['semester_count'] + 4
    else:
        return row['semester_count'] 

In [ ]:
df['semester_count_real'] = df.apply(get_true_sem_count, axis=1)

In [ ]:
def get_student_dropped_program(row):
    """
    dropout = 2 consecutive years of non-enrollment
    """
    if row['RETENTION_FLAG_AFTER_1_YEARS'] == 'N' and row['RETENTION_FLAG_AFTER_2_YEARS'] == 'N':
        return True
    elif row['RETENTION_FLAG_AFTER_2_YEARS'] == 'N' and row['RETENTION_FLAG_AFTER_3_YEARS'] == 'N':
        return True
    elif row['RETENTION_FLAG_AFTER_3_YEARS'] == 'N' and row['RETENTION_FLAG_AFTER_4_YEARS'] == 'N':
        return True
    else:
        return False

In [ ]:
def get_dropout_year(row):
    """
    dropout = 2 consecutive years of non-enrollment
    """
    if row['RETENTION_FLAG_AFTER_1_YEARS'] == 'N' and row['RETENTION_FLAG_AFTER_2_YEARS'] == 'N':
        return 1
    elif row['RETENTION_FLAG_AFTER_2_YEARS'] == 'N' and row['RETENTION_FLAG_AFTER_3_YEARS'] == 'N':
        return 2
    elif row['RETENTION_FLAG_AFTER_3_YEARS'] == 'N' and row['RETENTION_FLAG_AFTER_4_YEARS'] == 'N':
        return 3
    else:
        return np.nan

In [ ]:
df_cohort = pd.read_csv('../edw_askoski_student_cohorts_hashed.txt', sep='|')

df_cohort['student_dropped_program'] = df_cohort.apply(get_student_dropped_program, axis=1)

df_cohort['dropout_year'] = df_cohort.apply(get_dropout_year, axis=1)

df_cohort['dropout_semester_estimate'] = df_cohort['dropout_year']*2

df = df.merge(df_cohort[['ANON_ID', 'student_dropped_program', 'dropout_year', 'dropout_semester_estimate']],
              how='left', on='ANON_ID')

In [ ]:
def get_graduated_on_time(row):
    
    if row['is_transfer'] == 'Transfer':
        if row['YEARS_TO_GRADUATION'] <= 2:
            return True
        else:
            return False
    else:
        if row['YEARS_TO_GRADUATION'] <= 4:
            return True
        else:
            return False

In [ ]:
df['graduated_on_time'] = df.apply(get_graduated_on_time, axis=1)

# Course plot credit hours over predicted, colored by dropout composition

In [ ]:
df_plot = df_student_load[(df_student_load['SNAPSHOT_CODE']=='EOT') & (df_student_load['OFFERING_TYPE_DESC']=='Primary')]\
    [['ANON_ID', 'COURSE_SUBJECT_NAME_NUMBER', 'STUDENT_CREDIT_HRS_NBR', 'cl_combined_imputed', 'is_stem_course',
      'ACADEMIC_DEPARTMENT_NAME', 'ACADEMIC_DIVISION_NAME']]\
    .merge(df[['ANON_ID', 'graduated_on_time', 'student_dropped_program']], how='left', on='ANON_ID')\
    .drop_duplicates()\
    .groupby('COURSE_SUBJECT_NAME_NUMBER')\
    .agg({'STUDENT_CREDIT_HRS_NBR':'max', 
          'cl_combined_imputed':'mean', 
          'graduated_on_time':'mean', 
          'student_dropped_program': 'mean',
          'is_stem_course': 'max', 
          'ACADEMIC_DEPARTMENT_NAME': 'unique',
          'ACADEMIC_DIVISION_NAME': 'unique'})\
    .reset_index()\
    .dropna()
df_plot['ACADEMIC_DEPARTMENT_NAME'] = df_plot['ACADEMIC_DEPARTMENT_NAME'].map(lambda x: x[0])
df_plot['ACADEMIC_DIVISION_NAME'] = df_plot['ACADEMIC_DIVISION_NAME'].map(lambda x: x[0])
df_plot.head(3)

In [ ]:
def standardize_vector(v):
    mx = np.mean(v)
    sdx = np.std(v)
    return [(x-mx)/sdx for x in v]

In [ ]:
df_plot['ch_z'] = standardize_vector(df_plot['STUDENT_CREDIT_HRS_NBR'])
df_plot['cl_z'] = standardize_vector(df_plot['cl_combined_imputed'])
df_plot['diff_cl_ch'] = df_plot['cl_z'] - df_plot['ch_z'] #COURSE LOAD DISCREPANCY

In [ ]:
def get_standard_error(v, return_twice=True): 
    ans = np.std(v, ddof=1) / np.sqrt(np.size(v))
    return 2*ans if return_twice else ans

In [ ]:
df_plot\
    .groupby('is_stem_course')\
    [['diff_cl_ch']]\
    .agg([np.mean, get_standard_error])\
    .reset_index()

In [ ]:
import researchpy as rp

In [ ]:
summary, results = rp.ttest(group1= df_plot['diff_cl_ch'][df_plot['is_stem_course']], group1_name= "STEM",
                            group2= df_plot['diff_cl_ch'][~df_plot['is_stem_course']], group2_name= "Non-STEM")

In [ ]:
print(round(summary, 2))

In [ ]:
print(results)

In [ ]:
plt.rcParams.update({'font.size': 18})
fig = plt.figure(figsize=(16, 6), dpi=120)
df_plot['Top 10% Dropout Course'] =\
    df_plot['student_dropped_program'] > np.nanquantile(df_plot['student_dropped_program'], 0.9)

ax = sns.violinplot(x="is_stem_course", y="diff_cl_ch", hue='Top 10% Dropout Course', data=df_plot, split=True)
ax.set(ylim=(-6.5, None))
ax.legend(loc='upper center', title='Top 10% Dropout Course')

ax.set_xlabel('Course is a STEM Course')
ax.set_ylabel('Predicted Load - Credit Hours ($\Delta_{SD}$)')

plt.savefig('../plots/stem-non-stem-load-discrepancies-violin.pdf')

In [ ]:
len(list(df_plot[df_plot['diff_cl_ch']>2].COURSE_SUBJECT_NAME_NUMBER))

In [ ]:
# What features contributed to high discrepancy?

# Which features were most correlated with discrepancy?

In [ ]:
# Get features based on CID
fs = [
    '../research-data/processed/course-features-2017 Spring.csv',
    '../research-data/processed/course-features-2017 Fall.csv',
    '../research-data/processed/course-features-2018 Spring.csv',
    '../research-data/processed/course-features-2018 Fall.csv',
    '../research-data/processed/course-features-2019 Spring.csv',
    '../research-data/processed/course-features-2019 Fall.csv',
    '../research-data/processed/course-features-2020 Spring.csv',
    '../research-data/processed/course-features-2020 Fall.csv',
    '../research-data/processed/course-features-2021 Spring.csv'
]

remaining_cids = set(df_plot.COURSE_SUBJECT_NAME_NUMBER)
dfs = []
for f in fs[-1::-1]: # recent first
    tmp = pd.read_csv(f)
    dfs.append(tmp[tmp['course_name_number'].isin(remaining_cids)].copy())
    remaining_cids -= set(tmp.course_name_number)
    print(f'{len(remaining_cids)} courses remaining...')
    
course_features = pd.concat(dfs)

In [ ]:
df_plot_2 = df_plot.merge(course_features, how='left', 
                          left_on='COURSE_SUBJECT_NAME_NUMBER', right_on='course_name_number')

In [ ]:
from pingouin import partial_corr

In [ ]:
# Run partial correlations
def r_to_z(r):
    return math.log((1 + r) / (1 - r)) / 2.0

def z_to_r(z):
    e = math.exp(2 * z)
    return((e - 1) / (e + 1))

def r_confidence_interval(r, n, alpha=0.05):
    z = r_to_z(r)
    se = 1.0 / math.sqrt(n - 3)
    z_crit = scipy.stats.norm.ppf(1 - alpha/2)  # 2-tailed z critical value

    lo = z - z_crit * se
    hi = z + z_crit * se

    # Return a sequence
    return (z_to_r(lo), z_to_r(hi))

def correlation_pairwise_complete(series1, series2):
    x, y = series1.values, series2.values
    nas = np.logical_or(np.isnan(x), np.isnan(y))
    corr = scipy.stats.pearsonr(x[~nas], y[~nas])
    n = len(x[~nas])
    ci_low, ci_high = r_confidence_interval(corr[0], n, alpha=0.05)
    return corr, n, ci_low, ci_high

def partial_correlation_pairwise_complete(df: pd.DataFrame, series1: str, series2: str, controlseries: str):
    
    return partial_corr(data=df, x=series1, y=series2, covar=[controlseries], method='pearson')

def get_correlation_results_disc(filter_2sd=False):
    
    if filter_2sd:
        df_plot_3 = df_plot_2[df_plot_2['diff_cl_ch']>2].copy()
    else:
        df_plot_3 = df_plot_2.copy()
    
    feats, ns, cors, dfs, ps, ci_lows, ci_highs = [], [] ,[], [], [], [], []
    for col in set(tmp.columns) - {'course_name_number', 'section_num', 'secondary_section_number', 'all_section_numbers'}:
        try:
            (r, p), n, ci_low, ci_high = correlation_pairwise_complete(df_plot_3['diff_cl_ch'], df_plot_3[col])
        except:
            continue
        feats.append(col)
        ns.append(n)
        cors.append(r)
        dfs.append(n-2)
        ps.append(p)
        ci_lows.append(ci_low)
        ci_highs.append(ci_high)

    correlations = pd.DataFrame({
        'x': feats,
        'n': ns, 
        'r': [round(cor, 2) for cor in cors],
        'ci_low': [round(cor, 2) for cor in ci_lows],
        'ci_high': [round(cor, 2) for cor in ci_highs],
        'df': dfs,
        'p': [round(p, 3) for p in ps]
    }).sort_values(by='r', key=lambda x: -np.abs(x))

    correlations['is_c2v_var'] = correlations['x'].map(lambda s: 'c2v' in s)
    
    correlations['ci'] = correlations.apply(lambda row: '[' + str(row['ci_low']) + ', ' + str(row['ci_high']) + ']', axis=1)

    display(correlations[correlations['is_c2v_var']].head(15).sort_values(by='r', ascending=False))

    display(correlations[~correlations['is_c2v_var']].head(15).sort_values(by='r', ascending=False))
    return correlations

def get_partial_correlation_results_disc(filter_2sd=False):
    
    if filter_2sd:
        df_plot_3 = df_plot_2[df_plot_2['diff_cl_ch']>2].copy()
    else:
        df_plot_3 = df_plot_2.copy()
    
    res = []
    for col in set(df_plot_3.columns) - {'course_name_number', 'section_num', 'secondary_section_number', 'all_section_numbers'}:
        try:
            tmp = partial_correlation_pairwise_complete(df_plot_3, col, 'diff_cl_ch', 'ch_z')
            tmp['x'] = col
            res.append(tmp)
        except:
            continue

    ans = pd.concat(res)

    ans['is_c2v_var'] = ans['x'].map(lambda s: 'c2v' in s)
    
    ans['p-val'] = ans['p-val'].map(lambda x: round(x, 3))
    
    ans['r_abs'] = ans['r'].map(lambda x: np.abs(x))
    
    #display(ans[~ans['is_c2v_var']].head(15).sort_values(by='r', ascending=False))
    return ans

In [ ]:
cors1 = get_partial_correlation_results_disc()
print('##################################')
cors2 = get_partial_correlation_results_disc(filter_2sd=True)

In [ ]:
tab = cors1[~cors1['is_c2v_var']].sort_values(by='r_abs', ascending=False)
tab = tab[~tab['x'].isin(['STUDENT_CREDIT_HRS_NBR', 'cl_combined_imputed', 'cl_z', 'n_credit_hours'])]
tab.head(15).sort_values(by='r', ascending=False)

In [ ]:
cors1[cors1['x'].map(lambda s: 'prereq' in s and 'c2v' not in s)]

In [ ]:
c2v_cors = cors1[(~cors1['is_c2v_var']) & (cors1['x']!='cl_z') & (cors1['x']!='STUDENT_CREDIT_HRS_NBR') & (cors1['x']!='cl_combined_imputed')].sort_values(by='r_abs', ascending=False).head(15)

c2v_cors.sort_values(by='r', ascending=False)

In [ ]:
c2v_cors = cors1[(~cors1['is_c2v_var']) & (cors1['x']!='cl_z') & (cors1['x']!='STUDENT_CREDIT_HRS_NBR') & (cors1['x']!='cl_combined_imputed')].sort_values(by='r_abs', ascending=False).head(15)\
    .merge(cors2.add_suffix('_2sd'), how='left', left_on='x', right_on='x_2sd')

c2v_cors['2sd_robust'] = c2v_cors['p-val_2sd'].map(lambda x: x<.05)
c2v_cors.sort_values(by='r', ascending=False)

In [ ]:
df_plot_2.student_to_instructional_staff_ratio.mean()

In [ ]:
df_plot_2[df_plot_2['diff_cl_ch']>4].student_to_instructional_staff_ratio.mean()

In [ ]:
# Partial correlation with boostrap

In [ ]:
# Multiple R2 with boostrap c2v vs. c2v prereqs, controlling for credit hours

import statsmodels.api as sm
import statsmodels.formula.api as smf

from tqdm import tqdm

def get_r2_mr(df, c2v_prereq=False, return_model=False, include_credit_hours_only=False):
    
    tmp = df.copy()
    
    if c2v_prereq:
        tmp.replace([np.inf, -np.inf], np.nan, inplace=True)
        tmp.dropna(inplace=True, subset=[f'c2v_prereq_avg_{i}' for i in range(1, 300+1)] + ['ch_z'])
    
    X_cols = [f'c2v_prereq_avg_{i}' for i in range(1, 300+1)] if c2v_prereq else [f'c2v_{i}' for i in range(1, 300+1)]
    X_cols += ['ch_z']
    
    if include_credit_hours_only: 
        X_cols = ['ch_z']
    
    X = tmp[X_cols]
    y = tmp['diff_cl_ch']
            
    m = sm.OLS(y, X).fit()
    
    return m if return_model else m.rsquared_adj

In [ ]:
# R2 bootstrap, c2v course
n = 10 # reduced to 10 to run whole notebook faster
res = []
for _ in tqdm(range(n)):
    inds = np.random.randint(0, len(df_plot_2), size=len(df_plot_2))
    res.append(get_r2_mr(df_plot_2.iloc[inds], c2v_prereq=False, return_model=False))
print('C2V prereqs')
print(f"M = {np.mean(res)}, 95% CI = [{np.quantile(res, 0.025)}, {np.quantile(res, 0.975)}]")

In [ ]:
# R2 bootstrap, c2v prereqs
n = 10
res = []
for _ in tqdm(range(n)):
    inds = np.random.randint(0, len(df_plot_2), size=len(df_plot_2))
    res.append(get_r2_mr(df_plot_2.iloc[inds], c2v_prereq=True, return_model=False))
print('C2V prereqs')
print(f"M = {np.mean(res)}, 95% CI = [{np.quantile(res, 0.025)}, {np.quantile(res, 0.975)}]")

In [ ]:
# R2 bootstrap, contribution of credit hours only
n = 10
res = []
for _ in tqdm(range(n)):
    inds = np.random.randint(0, len(df_plot_2), size=len(df_plot_2))
    res.append(get_r2_mr(df_plot_2.iloc[inds], c2v_prereq=False, return_model=False, include_credit_hours_only=True))
print('C2V prereqs')
print(f"M = {np.mean(res)}, 95% CI = [{np.quantile(res, 0.025)}, {np.quantile(res, 0.975)}]")

In [ ]:
# Just with top courses
df_plot_3 = df_plot_2[df_plot_2['diff_cl_ch']>2].copy()

In [ ]:
# R2 bootstrap, c2v course
n = 10
res = []
for _ in tqdm(range(n)):
    inds = np.random.randint(0, len(df_plot_3), size=len(df_plot_3))
    res.append(get_r2_mr(df_plot_3.iloc[inds], c2v_prereq=False, return_model=False))
print('C2V prereqs')
print(f"M = {np.mean(res)}, 95% CI = [{np.quantile(res, 0.025)}, {np.quantile(res, 0.975)}]")

In [ ]:
# R2 bootstrap, c2v prereqs
n = 10 
res = []
for _ in tqdm(range(n)):
    inds = np.random.randint(0, len(df_plot_3), size=len(df_plot_3))
    res.append(get_r2_mr(df_plot_3.iloc[inds], c2v_prereq=True, return_model=False))
print('C2V prereqs')
print(f"M = {np.mean(res)}, 95% CI = [{np.quantile(res, 0.025)}, {np.quantile(res, 0.975)}]")

In [ ]:
# R2 bootstrap, contribution of credit hours only
n = 10
res = []
for _ in tqdm(range(n)):
    inds = np.random.randint(0, len(df_plot_3), size=len(df_plot_3))
    res.append(get_r2_mr(df_plot_3.iloc[inds], c2v_prereq=False, return_model=False, include_credit_hours_only=True))
print('C2V prereqs')
print(f"M = {np.mean(res)}, 95% CI = [{np.quantile(res, 0.025)}, {np.quantile(res, 0.975)}]")

In [ ]:
# Prerequsitie adjustment

# Compute how much predicted load is in one prereq -> regression of cl over n prereqs

# Multiply by modal number of satisfied prereqs

# Convert into credit hours

In [ ]:
# Input
fs = [
    '../research-data/processed/course-features-2017 Spring.csv',
    '../research-data/processed/course-features-2017 Fall.csv',
    '../research-data/processed/course-features-2018 Spring.csv',
    '../research-data/processed/course-features-2018 Fall.csv',
    '../research-data/processed/course-features-2019 Spring.csv',
    '../research-data/processed/course-features-2019 Fall.csv',
    '../research-data/processed/course-features-2020 Spring.csv',
    '../research-data/processed/course-features-2020 Fall.csv',
    '../research-data/processed/course-features-2021 Spring.csv'
]

# Sems
refs = [f.split('-')[-1].split('.')[0] for f in fs]

dfs = [] 
for f, sem in zip(fs, refs):
    tmp = pd.read_csv(f)
    tmp = tmp[['course_name_number', 'n_prereqs', 'n_satisfied_prereqs_all_past_semesters']]
    tmp = tmp.drop_duplicates()
    tmp = tmp.rename(columns = {'course_name_number': 'COURSE_SUBJECT_NAME_NUMBER'})
    dfs.append(tmp)
df_prereqs = pd.concat(dfs)

In [ ]:
df_plot_2 = df_plot[df_plot['diff_cl_ch'] > 5].copy()

In [ ]:
df_plot_2 = df_plot[df_plot['is_stem_course']].copy()

In [ ]:
df_plot_2 = df_plot[df_plot['diff_cl_ch'] > 3].copy()

In [ ]:
df_prereqs_load = df_plot_2[['COURSE_SUBJECT_NAME_NUMBER', 'cl_combined_imputed']]\
    .merge(df_prereqs, on='COURSE_SUBJECT_NAME_NUMBER', how='left')\
    .drop_duplicates()

In [ ]:
import statsmodels.formula.api as sm

tmp = df_prereqs_load.copy()
#tmp['cl_combined_imputed'] = np.log(tmp['cl_combined_imputed'])
#tmp['n_prereqs'] = np.log(tmp['n_prereqs'])

result = sm.ols(formula="cl_combined_imputed ~ n_satisfied_prereqs_all_past_semesters", data=tmp).fit()
print(result.params)

In [ ]:
df_prereqs_load['n_satisfied_prereqs_all_past_semesters'].mean()

In [ ]:
# three credit hour course -> 2.62 analytics units
2.62/3

In [ ]:
# Re-scale adjustment to credit hour scale
(0.446950*0.7802066342154884)/0.8733333333333334

# Load distribution longitudinal

In [ ]:
# Figure 3 Freshman vs. advanced part 1
df_sem_load_plot = df.groupby(['semester_count', 'is_stem_major_student'])['sem_load_credit_hours'].agg([np.mean, get_standard_error]).reset_index()

matplotlib.rcParams.update({'font.size': 24})
fig,ax = plt.subplots(figsize=(14,8), dpi=180)
lvls = df_sem_load_plot.is_stem_major_student.unique()
first = True
for i in lvls:
    ax.errorbar(x = df_sem_load_plot[df_sem_load_plot['is_stem_major_student']==i]["semester_count"],
                y=df_sem_load_plot[df_sem_load_plot['is_stem_major_student']==i]["mean"], 
                yerr=df_sem_load_plot[df_sem_load_plot['is_stem_major_student']==i]["get_standard_error"],
                label=i, linestyle='dashed' if not first else 'solid')
    first = False
ax.legend()
ax.set_xlabel('Semester Count')
ax.set_ylabel('Average Credit Hour Load')
ax.legend(labels=['Non-STEM Major Students', 'STEM Major Students'])

fig.savefig('../plots/avg-credit-load-stem-non-stem.pdf')

In [ ]:
# Figure 3 Freshman vs. advanced part 1
df_sem_load_plot = df.groupby(['semester_count', 'is_stem_major_student'])['sem_load_predicted'].agg([np.mean, get_standard_error]).reset_index()

matplotlib.rcParams.update({'font.size': 24})
fig, ax = plt.subplots(figsize=(14,8), dpi=180)
lvls = df_sem_load_plot.is_stem_major_student.unique()
first = True
for i in lvls:
    ax.errorbar(x = df_sem_load_plot[df_sem_load_plot['is_stem_major_student']==i]["semester_count"],
                y=df_sem_load_plot[df_sem_load_plot['is_stem_major_student']==i]["mean"], 
                yerr=df_sem_load_plot[df_sem_load_plot['is_stem_major_student']==i]["get_standard_error"],
                label=i, linestyle='dashed' if not first else 'solid')
    first = False
ax.legend()
ax.set_xlabel('Semester Count')
ax.set_ylabel('Average Predicted Course Load')
ax.legend(labels=['Non-STEM Major Students', 'STEM Major Students'])

ax.yaxis.set_ticks(np.arange(10.25, 12.75, 0.25))

fig.savefig('../plots/avg-pred-load-stem-non-stem.pdf')

In [ ]:
# Figure 3 Freshman vs. advanced part 1
df_sem_load_plot = df.groupby(['semester_count', 'is_transfer'])['sem_load_predicted'].agg([np.mean, get_standard_error]).reset_index()

fig,ax = plt.subplots(figsize=(14,8))
lvls = df_sem_load_plot.is_transfer.unique()
for i in lvls:
    ax.errorbar(x = df_sem_load_plot[df_sem_load_plot['is_transfer']==i]["semester_count"],
                y=df_sem_load_plot[df_sem_load_plot['is_transfer']==i]["mean"], 
                yerr=df_sem_load_plot[df_sem_load_plot['is_transfer']==i]["get_standard_error"],label=i)
ax.legend()

In [ ]:
def standardize_vector(v):
    mx = np.mean(v)
    sdx = np.std(v)
    return [(x-mx)/sdx for x in v]

df['sem_load_predicted_z'] = standardize_vector(df['sem_load_predicted'])
df['sem_load_credit_hours_z'] = standardize_vector(df['sem_load_credit_hours'])
df['sem_load_diff_pred_minus_ch'] = df['sem_load_predicted_z'] - df['sem_load_credit_hours_z']

df.sort_values(by='sem_load_diff_pred_minus_ch')

In [ ]:
# Export to conduct likelihood-ratio tests in R
df.to_csv('../research-data/processed/lak23wlsample.csv', index=False)

## Logistic regression model inferences dropout and on-time graduation

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# defining the dependent and independent variables
tmp = df[['sem_load_credit_hours', 'sem_load_predicted', 'student_dropped_program']].dropna().copy()
Xtrain = tmp[['sem_load_credit_hours', 'sem_load_predicted']]
ytrain = tmp[['student_dropped_program']]
print(tmp.shape)
    
# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()

print(log_reg.summary())

y = np.array(Xtrain['sem_load_credit_hours'])
x = np.array(Xtrain['sem_load_predicted'])
z = np.array(ytrain['student_dropped_program'])

x_pred = np.linspace(0, 35, 100)      # range of porosity values
y_pred = np.linspace(0, 35, 100)   # range of VR values
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([yy_pred.flatten(), xx_pred.flatten()]).T
predicted = log_reg.predict(model_viz)

In [ ]:
def get_predictions_xcp(q=.5):
    """
    Get predictions based on a particular credit hour quantile.
    x -> predicted, fix -> credit hour semester load
    Validated
    """
    x_pred = np.linspace(0, 30, 100)      # predicted load
    y_pred = np.array([np.quantile(Xtrain['sem_load_credit_hours'], q)]*100)      # fixed credit hours
    xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
    model_viz = np.array([yy_pred.flatten(), xx_pred.flatten()]).T
    predicted = log_reg.predict(model_viz)
    x_plot = model_viz[:,1] # predicted load
    y_plot = predicted
    return x_plot, y_plot

In [ ]:
def get_predictions_xch(q=.5):
    """
    Get predictions based on a particular credit hour quantile.
    x -> credit hours, fix -> predicted load
    """
    y_pred = np.linspace(0, 30, 100)      # credit hour load
    x_pred = np.array([np.quantile(Xtrain['sem_load_predicted'], q)]*100)      # fixed predicted load
    #xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
    model_viz = np.array([y_pred.flatten(), x_pred.flatten()]).T
    predicted = log_reg.predict(model_viz)
    x_plot = model_viz[:,0] # credit hour load
    y_plot = predicted
    return x_plot, y_plot

In [ ]:
y_pred = np.linspace(0, 30, 100)      # credit hour load
x_pred = np.array([np.quantile(Xtrain['sem_load_predicted'], .5)]*100)      # fixed predicted load
#xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([y_pred.flatten(), x_pred.flatten()]).T
predicted = log_reg.predict(model_viz)
x_plot = model_viz[:,0] # credit hour load
y_plot = predicted

In [ ]:
# New dropout model space model 2d by median split
import matplotlib.cm as cm

plt.figure(figsize=(8, 6), dpi=80)

x_plot, _ = get_predictions_xcp()
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
CB_color_cycle.reverse()
CB_color_cycle[:len(x_plot)]
the_range = list(range(10, 100, 20))
for q, c in zip(the_range, CB_color_cycle):
    _, y_plot = get_predictions_xcp(q/100)
    ch = int(np.quantile(Xtrain['sem_load_credit_hours'], q/100))
    plt.annotate(f"Q{q} ({ch} CH)", (31, max(y_plot)), size=12)
    plt.scatter(x_plot, y_plot, color=c)
plt.xlabel('Predicted Semester Load')
plt.ylabel('P(Dropout)')
plt.xlim(0, 38)
plt.yticks([0.025, 0.05, 0.075, 0.1, 0.125])
plt.savefig('../plots/predicted-marginal-dropout.png', bbox_inches='tight')
plt.show()

In [ ]:
# Model Space On-Time Graduation

# defining the dependent and independent variables
tmp = df[['sem_load_credit_hours', 'sem_load_predicted', 'graduated_on_time']].dropna().copy()
tmp['graduated_on_time'] = tmp['graduated_on_time'].map(lambda x: 0 if x == True else 1)
Xtrain = tmp[['sem_load_credit_hours', 'sem_load_predicted']]
ytrain = tmp[['graduated_on_time']]
print(tmp.shape)

# building the model and fitting the data
log_reg = smf.logit(formula='graduated_on_time ~ sem_load_predicted + sem_load_credit_hours + sem_load_predicted:sem_load_credit_hours', data=tmp).fit()

print(log_reg.summary())

y = np.array(Xtrain['sem_load_credit_hours'])
x = np.array(Xtrain['sem_load_predicted'])
z = np.array(ytrain['graduated_on_time'])


In [ ]:
def get_predictions_xch(q=.5):
    """
    Get predictions based on a particular credit hour quantile.
    x -> credit hours, fix -> predicted load
    """
    y_pred = np.linspace(0, 30, 100)      # credit hour load
    x_pred = np.array([np.quantile(Xtrain['sem_load_predicted'], q)]*100)      # fixed predicted load
    #xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
    model_viz = np.array([y_pred.flatten(), x_pred.flatten()]).T
    predicted = np.array(log_reg.predict(pd.DataFrame({'sem_load_credit_hours': y_pred.flatten(), 'sem_load_predicted': x_pred.flatten()})))
    x_plot = model_viz[:,0] # credit hour load
    y_plot = predicted
    return x_plot, y_plot

In [ ]:
def get_predictions_xcp(q=.5):
    """
    Get predictions based on a particular credit hour quantile.
    x -> predicted, fix -> credit hour semester load
    Validated
    """
    x_pred = np.linspace(0, 30, 100)      # predicted load
    y_pred = np.array([np.quantile(Xtrain['sem_load_credit_hours'], q)]*100)      # fixed credit hours
    xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
    model_viz = np.array([yy_pred.flatten(), xx_pred.flatten()]).T
    predicted = np.array(log_reg.predict(pd.DataFrame({'sem_load_credit_hours': yy_pred.flatten(), 'sem_load_predicted': xx_pred.flatten()})))
    x_plot = model_viz[:,1] # predicted load
    y_plot = predicted
    return x_plot, y_plot

In [ ]:
# New dropout model space model 2d by median split
import matplotlib.cm as cm

x_plot, _ = get_predictions_xch()
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
CB_color_cycle.reverse()
CB_color_cycle[:len(x_plot)]
the_range = list(range(10, 100, 10))
for q, c in zip(the_range, CB_color_cycle):
    _, y_plot = get_predictions_xch(q/100)
    plt.scatter(x_plot, y_plot, color=c)
plt.xlabel('Credit Hour Semester Load')
plt.show()

In [ ]:
# New dropout model space model 2d by median split
import matplotlib.cm as cm

plt.figure(figsize=(8, 6), dpi=80)

x_plot, _ = get_predictions_xcp()
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
CB_color_cycle.reverse()
CB_color_cycle[:len(x_plot)]
the_range = list(range(10, 100, 20))
for q, c in zip(the_range, CB_color_cycle):
    _, y_plot = get_predictions_xcp(q/100)
    ch = int(np.quantile(Xtrain['sem_load_credit_hours'], q/100))
    plt.annotate(f"Q{q} ({ch} CH)", (-7, min(y_plot)), size=12)
    plt.scatter(x_plot, y_plot, color=c)
plt.xlabel('Predicted Semester Load')
plt.ylabel('P(Delayed Graduation)')
plt.xlim(-7.5, 30)
plt.yticks([0.1, 0.15, 0.2, 0.25, 0.3, 0.35])
plt.savefig('../plots/predicted-marginal-delay.png', bbox_inches='tight')
plt.show()

## Empirically derived cutoff for credit hour load or semester load

In [ ]:
# building the model and fitting the data
log_reg = smf.logit(formula='graduated_on_time ~ sem_load_predicted', data=tmp).fit()
print(log_reg.summary())

In [ ]:
three_ch_median_cl = df_plot[df_plot['STUDENT_CREDIT_HRS_NBR']==3].cl_combined_imputed.median()
3*(25/three_ch_median_cl)

In [ ]:
three_ch_median_cl

In [ ]:
ytrain.value_counts()

In [ ]:
round(12568/(45757+12568), 4)*100

## Fitted model performance

In [ ]:
from sklearn.metrics import roc_auc_score
import random
from tqdm import tqdm

# ROC bootstrap
n = 1000
res = []
for _ in tqdm(range(n)):
    
    inds = np.random.randint(0, len(Xtrain), size=len(Xtrain))
    xt = Xtrain.iloc[inds]
    yt = ytrain.iloc[inds]

    res.append(roc_auc_score(yt, log_reg.predict(xt)))

print(f'M = {np.mean(res)} , 95% CI = [{np.quantile(res, 0.025)} , {np.quantile(res, 0.975)}]')

In [ ]:
# Additive model only credit hours comparison

# defining the dependent and independent variables
tmp = df[['sem_load_credit_hours', 'sem_load_predicted', 'student_dropped_program']].dropna().copy()
Xtrain = tmp[['sem_load_credit_hours']]
ytrain = tmp[['student_dropped_program']]
    
# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()

# ROC bootstrap
n = 1000
res = []
for _ in tqdm(range(n)):
    
    inds = np.random.randint(0, len(Xtrain), size=len(Xtrain))
    xt = Xtrain.iloc[inds]
    yt = ytrain.iloc[inds]

    res.append(roc_auc_score(yt, log_reg.predict(xt)))

print(f'M = {np.mean(res)} , 95% CI = [{np.quantile(res, 0.025)} , {np.quantile(res, 0.975)}]')

In [ ]:
# Model Space Dropout

import statsmodels.api as sm
import statsmodels.formula.api as smf

# defining the dependent and independent variables
tmp = df[['sem_load_credit_hours', 'sem_load_predicted', 'student_dropped_program']].dropna().copy()
Xtrain = tmp[['sem_load_credit_hours', 'sem_load_predicted']]
ytrain = tmp[['student_dropped_program']]
print(tmp.shape)
    
# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()

print(log_reg.summary())

y = np.array(Xtrain['sem_load_credit_hours'])
x = np.array(Xtrain['sem_load_predicted'])
z = np.array(ytrain['student_dropped_program'])

x_pred = np.linspace(0, 35, 100)      # range of porosity values
y_pred = np.linspace(0, 35, 100)   # range of VR values
xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)
model_viz = np.array([yy_pred.flatten(), xx_pred.flatten()]).T
predicted = log_reg.predict(model_viz)

In [ ]:
fig = plt.figure(figsize=(14, 14))
ax = plt.axes(projection='3d')

fig.tight_layout(pad=2)

ax.plot(y, x, z, color='k', zorder=15, linestyle='none', marker='o', alpha=0.04)
ax.scatter(yy_pred.flatten(), xx_pred.flatten(), predicted, facecolor=(0,0,0,0), s=20, edgecolor='#70b3f0')
ax.set_xlabel('Credit Hour Semester Load', fontsize=24)
ax.set_ylabel('Predicted Semester Load', fontsize=24)
ax.set_zlabel('P(Stop-Out)', fontsize=24)
ax.locator_params(nbins=4, axis='x')
ax.locator_params(nbins=5, axis='x')

ax.set_xticks([0, 10, 20, 30, 40])
ax.set_yticks([0, 10, 20, 30, 40])
ax.set_zticks([0, .5, 1])

ax.set_xticklabels([0, 10, 20, 30, 40], fontsize=24)
ax.set_yticklabels([0, 10, 20, 30, 40], fontsize=24)
ax.set_zticklabels([0, .5, 1], fontsize=24)

ax.xaxis.labelpad=15
ax.yaxis.labelpad=15 
ax.zaxis.labelpad=7.5

plt.savefig("../plots/dropout-model-space.png")

In [ ]:
# Model Space On-Time Graduation

# defining the dependent and independent variables
tmp = df[['sem_load_credit_hours', 'sem_load_predicted', 'graduated_on_time']].dropna().copy()
tmp['graduated_on_time'] = tmp['graduated_on_time'].map(lambda x: 0 if x == True else 1)
Xtrain = tmp[['sem_load_credit_hours', 'sem_load_predicted']]
ytrain = tmp[['graduated_on_time']]
print(tmp.shape)

# building the model and fitting the data
log_reg = smf.logit(formula='graduated_on_time ~ sem_load_predicted + sem_load_credit_hours + sem_load_predicted:sem_load_credit_hours', data=tmp).fit()

print(log_reg.summary())

y = np.array(Xtrain['sem_load_credit_hours'])
x = np.array(Xtrain['sem_load_predicted'])
z = np.array(ytrain['graduated_on_time'])

x_pred = np.linspace(0, 35, 100)      # range of porosity values
y_pred = np.linspace(0, 35, 100)   # range of VR values

xx_pred, yy_pred = np.meshgrid(x_pred, y_pred)

predicted = log_reg.predict(pd.DataFrame({'sem_load_credit_hours': yy_pred.flatten(), 'sem_load_predicted': xx_pred.flatten()}))

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot(y, x, z, color='k', zorder=15, linestyle='none', marker='o', alpha=0.01)
ax.scatter(yy_pred.flatten(), xx_pred.flatten(), predicted, facecolor=(0,0,0,0), s=20, edgecolor='#70b3f0')
ax.set_xlabel('Credit Hour Semester Load', fontsize=12)
ax.set_ylabel('Predicted Semester Load', fontsize=12)
ax.set_zlabel('Delayed Graduation', fontsize=12)
ax.locator_params(nbins=4, axis='x')
ax.locator_params(nbins=5, axis='x')
fig.tight_layout()

ax.set_xticks([0, 10, 20, 30, 40])
ax.set_yticks([0, 10, 20, 30, 40])
ax.set_zticks([0, .5, 1])

ax.set_xticklabels([0, 10, 20, 30, 40], fontsize=14)
ax.set_yticklabels([0, 10, 20, 30, 40], fontsize=14)
ax.set_zticklabels([0, .5, 1], fontsize=14)

plt.savefig("../plots/on-time-graduation-model-space.png", bbox_inches='tight')

In [ ]:
from sklearn.metrics import roc_auc_score
import random
from tqdm import tqdm

# ROC bootstrap
n = 1000
res = []
for _ in tqdm(range(n)):
    
    inds = np.random.randint(0, len(Xtrain), size=len(Xtrain))
    xt = Xtrain.iloc[inds]
    yt = ytrain.iloc[inds]

    res.append(roc_auc_score(yt, log_reg.predict(xt)))

print(f'M = {np.mean(res)} , 95% CI = [{np.quantile(res, 0.025)} , {np.quantile(res, 0.975)}]')

In [ ]:
# Additive model only credit hours comparison

# defining the dependent and independent variables
tmp = df[['sem_load_credit_hours', 'sem_load_predicted', 'graduated_on_time']].dropna().copy()
tmp['graduated_on_time'] = tmp['graduated_on_time'].map(lambda x: 0 if x == True else 1)
Xtrain = tmp[['sem_load_credit_hours', 'sem_load_predicted']]
ytrain = tmp[['graduated_on_time']]

# building the model and fitting the data
log_reg = smf.logit(formula='graduated_on_time ~ sem_load_credit_hours', data=tmp).fit()

    
# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()

# ROC bootstrap
n = 1000
res = []
for _ in tqdm(range(n)):
    
    inds = np.random.randint(0, len(Xtrain), size=len(Xtrain))
    xt = Xtrain.iloc[inds]
    yt = ytrain.iloc[inds]

    res.append(roc_auc_score(yt, log_reg.predict(xt)))

print(f'M = {np.mean(res)} , 95% CI = [{np.quantile(res, 0.025)} , {np.quantile(res, 0.975)}]')